In [1]:
import glob
import hist.util
import cupy as cp
import numpy as np
from datetime import datetime
from hist._cmd_line_util import open_input_image
from hist.classical import histeq, histeq_trans, histeq_apply, __as_unsigned
from utils import clear_line


# 2D and 3D Images
images = glob.glob('../c4l-image-dataset/*/*')
n_imgs = len(images)
for i, filename in enumerate(images):
    string = f'Progress: {i}/{n_imgs}\tCurrent image: {filename[21:]}'
    print(string, end='')

    # Setup for function calls
    timing_data = []
    img_cpu = open_input_image(filename)
    img_gpu = cp.array(img_cpu)
    h_dst = 64 # Default for histeq function
    
    
    # GPU timing
    img_gpu_u, orig_dt = __as_unsigned(img_gpu)
    h_src_gpu = hist.util.__imhist(img_gpu_u)
    
    data = %timeit -q -o histeq_trans(h_src_gpu, img_gpu_u.dtype)
    timing_data.append(data)
    transform_gpu = histeq_trans(h_src_gpu, h_dst, img_gpu_u.dtype)
    
    data = %timeit -q -o histeq_apply(img_gpu_u, transform_gpu)
    timing_data.append(data)
    
    data = %timeit -q -o histeq(img_gpu, h_dst)
    timing_data.append(data)
    
    
    # CPU timing
    img_cpu_u, orig_dt = __as_unsigned(img_cpu)
    h_src_cpu = __imhist(img_cpu_u)
    
    data = %timeit -q -o histeq_trans(h_src_cpu, h_dst, img_cpu_u.dtype)
    timing_data.append(data)
    transform_cpu = histeq_trans(h_src_cpu, h_dst, img_cpu_u.dtype)
    
    data = %timeit -q -o histeq_apply(img_cpu_u, transform_cpu)
    timing_data.append(data)
    
    data = %timeit -q -o histeq(img_cpu, h_dst)
    timing_data.append(data)
    
    # Save data in file; appends to the end of the file
    with open('classical.txt', 'a+') as file:
        file.write('\n\n' + filename + '\n')
        for data in timing_data:
            file.write(str(data) + '\n')

    clear_line(len(string)) # Clears the current output

print('Finished!')


Finished!


In [ ]:
import os
import os.path
from utils import prefix_to_micro


if os.path.exists('classical.txt'):
    with open('classical.txt', 'r') as temp:
        with open('classical_timing.csv', 'w+') as file:
            file.write('Image,histeq_trans_gpu Mean,histeq_trans_gpu Std. Dev.,histeq_apply_gpu Mean,'
                       'histeq_apply_gpu Std. Dev.,histeq_gpu Mean,histeq_gpu Std. Dev.,'
                       'histeq_trans_cpu Mean,histeq_trans_cpu Std. Dev.,histeq_apply_cpu Mean,'
                       'histeq_apply_cpu Std. Dev.,histeq_cpu Mean,histeq_cpu Std. Dev.\n')
            data = temp.read().strip().split('\n\n')
            for img in data:
                img = img.strip().split('\n')
                data_list = [img[0].strip('..').strip('/')]
                # Get time values and convert them to microseconds
                for i in range(1, 7):
                    vals = img[i].split(' ')
                    data_list.append(float(vals[0])*prefix_to_micro(vals[1]))
                    data_list.append(float(vals[3])*prefix_to_micro(vals[4]))
                file.write(','.join([str(_) for _ in data_list]) + '\n')
            
#os.remove("classical.txt")
